In [62]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, MaxPooling1D, Flatten, Embedding, Input
from keras.layers.convolutional import Conv1D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard, EarlyStopping
from tensorboard.plugins.pr_curve import summary as pr_summary

%matplotlib inline

# Data Exploration

In [63]:
data = pd.read_csv('../../datasets/heli_uniform/heli_1m_uniform.csv')

In [ ]:
data.head()

In [ ]:
data.hist(grid=False, figsize=(15,40), layout=(10,3), bins=100, sharex=True)

The data is imbalanced with more positive examples.

In [ ]:
X = data.drop('label', axis=1)
y = data['label']
(num_samples, num_features) = X.shape
print (num_samples, num_features)

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, shuffle=True)

# TensorBoard Visualization

Displays Precision-Recall curve for Keras models.

Adapted from: https://medium.com/@akionakas/precision-recall-curve-with-keras-cd92647685e1

In [74]:
class PRTensorBoard(TensorBoard):
    def __init__(self, *args, **kwargs):
        # One extra argument to indicate whether or not to use the PR curve summary.
        self.pr_curve = kwargs.pop('pr_curve', True)
        super(PRTensorBoard, self).__init__(*args, **kwargs)

        global tf
        import tensorflow as tf

    def set_model(self, model):
        super(PRTensorBoard, self).set_model(model)

        if self.pr_curve:
            # Get the prediction and label tensor placeholders.
            predictions = self.model._feed_outputs[0]
            labels = tf.cast(self.model._feed_targets[0], tf.bool)
            # Create the PR summary OP.
            self.pr_summary = pr_summary.op(name='pr_curve',
                                            predictions=predictions,
                                            labels=labels,
                                            display_name='Precision-Recall Curve')

    def on_epoch_end(self, epoch, logs=None):
        super(PRTensorBoard, self).on_epoch_end(epoch, logs)

        if self.pr_curve and self.validation_data:
            # Get the tensors again.
            tensors = self.model._feed_targets + self.model._feed_outputs
            # Predict the output.
            predictions = self.model.predict(self.validation_data[0])
            # Build the dictionary mapping the tensor to the data.
            val_data = [self.validation_data[1], predictions]
            feed_dict = dict(zip(tensors, val_data))
            # Run and add summary.
            result = self.sess.run([self.pr_summary], feed_dict=feed_dict)
            self.writer.add_summary(result[0], epoch)
        self.writer.flush()

# DNN Model

In [66]:
epochs = 10
batch_size = 32

In [ ]:
model = Sequential()

model.add(Dense(16, input_shape=(num_features,)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(16))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(16))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss = 'binary_crossentropy',
              optimizer = opt,
              metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,
          shuffle=True)

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size)
print model.metrics_names
print score

# CNN Model

In [67]:
epochs = 10
batch_size = 32
X_train_cnn = np.expand_dims(X_train.as_matrix(), axis=2)
X_test_cnn = np.expand_dims(X_test.as_matrix(), axis=2)

In [79]:
# 16 3x1 convolutions
inputs = Input(shape=(num_features, 1))
conv1 = Conv1D(16, 3, padding='same', strides=1)(inputs)
bn1 = BatchNormalization()(conv1)
relu1 = Activation('relu')(bn1)

# 32 3x16 convolutions
conv2 = Conv1D(32, 3, padding='same', strides=1)(relu1)
bn2 = BatchNormalization()(conv2)
relu2 = Activation('relu')(bn2)
flatten = Flatten()(relu2)

# FC 2048
fc1 = Dense(2048)(flatten)
bn3 = BatchNormalization()(fc1)
relu3 = Activation('relu')(bn3)
drop1 = Dropout(0.5)(relu3)

# FC 1024
fc2 = Dense(1024)(drop1)
bn4 = BatchNormalization()(fc2)
relu4 = Activation('relu')(bn4)
drop2 = Dropout(0.5)(relu4)

# FC 512
fc3 = Dense(512)(drop2)
bn5 = BatchNormalization()(fc3)
relu5 = Activation('relu')(bn5)
drop3 = Dropout(0.5)(relu5)

# Output
fc4 = Dense(1)(drop3)
outputs = Activation('sigmoid')(fc4)

model = Model(inputs=inputs, outputs=outputs)

In [80]:
opt = keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

model.compile(loss = 'binary_crossentropy',
              optimizer = opt,
              metrics = ['accuracy'])

In [82]:
model.fit(X_train_cnn, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=[X_test_cnn, y_test], # NOTE: must pass test data for PRTensorBoard callback
          shuffle=True,
          callbacks=[PRTensorBoard(log_dir='logs/nadam_dropout50'), EarlyStopping(monitor='val_acc', patience=2)])

Train on 800000 samples, validate on 200000 samples
Epoch 1/10
800000/800000 [==============================] - 2868s 4ms/step - loss: 0.2930 - acc: 0.8717 - val_loss: 0.1820 - val_acc: 0.9223
Epoch 2/10
800000/800000 [==============================] - 3693s 5ms/step - loss: 0.2261 - acc: 0.9029 - val_loss: 0.1575 - val_acc: 0.9355
Epoch 3/10
800000/800000 [==============================] - 2820s 4ms/step - loss: 0.2066 - acc: 0.9120 - val_loss: 0.1501 - val_acc: 0.9396
Epoch 4/10
800000/800000 [==============================] - 3235s 4ms/step - loss: 0.1942 - acc: 0.9177 - val_loss: 0.1279 - val_acc: 0.9488
Epoch 5/10
800000/800000 [==============================] - 3461s 4ms/step - loss: 0.1854 - acc: 0.9218 - val_loss: 0.1280 - val_acc: 0.9492
Epoch 6/10
800000/800000 [==============================] - 3518s 4ms/step - loss: 0.1788 - acc: 0.9247 - val_loss: 0.1238 - val_acc: 0.9505
Epoch 7/10
800000/800000 [==============================] - 3099s 4ms/step - loss: 0.1742 - acc: 0.926

In [ ]:
score = model.evaluate(X_test_cnn, y_test, batch_size=batch_size)
print model.metrics_names
print score

In [83]:
model.save('models/cnn_16_32_dropout50_nadam.h5')